In [8]:
#Imports
import json
import numpy
import sklearn 
#Base Naive Bayes classifier:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from evaluation import compute_macro_f1_score
from evaluation import compute_micro_f1_score
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import re

In [9]:
import json
with open('data/train.json', 'r', encoding='utf-8') as fp:
        train_data = json.load(fp)
print(len(train_data ))

with open('data/valid.json', 'r', encoding='utf-8') as fp:
        valid_old = json.load(fp)

with open('data/valid_new.json', 'r', encoding='utf-8') as fp:
        valid_new = json.load(fp)
def get_text_langid(data):
        X = ['^' + i['text'] + '$' for i in data]
        y = [i['langid'] for i in data]
        return X, y
X_valid_old, y_valid_old = get_text_langid(valid_old)
X_valid_new, y_valid_new = get_text_langid(valid_new)
X_train, y_train = get_text_langid(train_data)
# X_valid_old_cnt = vec.transform(X_valid_old)
# X_valid_new_cnt = vec.transform(X_valid_new)

800003


## Setting up the predicted variable (language)

In [10]:
languages = set();
for i in train_data:
    languages.add(i['langid'])

languages = list(languages); #convert it into a list.
langind = {};
for i in languages:
    langind[i] = len(langind)
langname = {};
for i in langind:
    langname[langind[i]] = i;

### Applying a Simple NaiveBayes classifier at the start

In [59]:
def split_by_space(text): #A global function for consistent tokenization.
    return text[1:-1].lower().split(); #simple for now.

def split_by_punctuation(text):
    t = re.split(' |;|\*|\n|:|@|#|!|\(|\)', text[1:-1]);
    if(t[-1] == ''):
        t = t[:-1];
    return t;

#So we cannot do the bigram testing SINCE bigrams would be too big to handle. but we can maybe try tho

def unigram(text):
    return [x for x in text];
def quadgram(text):
    s = text.lower();
    return [text[i:i+4].lower() for i in range(len(text)-3)];
def pentagram(text):
    s = text.lower();
    return [text[i:i+5] for i in range(len(text)-4)];
def word_and_quadgram(text):
    s = text.lower();
    return split_by_space(s) + quadgram(s);
def word_and_pentagram(text):
    s = text.lower();
    return split_by_punctuation(s) + pentagram(s);
pentauniparameter = 1;
def penta_and_unigram(text):
    s = text.lower();
    return pentauniparameter*pentagram(s) + unigram(s);

def get_vocab(data):
    vocab = set();
    for i in range(len(data)):
        strlist = unigram(data[i]['text']); #The string we need to determine the language of.
        for j in strlist:
            if(j == 'ằ'):
                print('found here: ', data[i]);
            vocab.add(j); #we add all the words to the vocabulary.
    return vocab
vocab = get_vocab(train_data)
split_by_punctuation("^hello world(yolo)$")

found here:  {'text': 'Fellow #Vietnam activists preparing to attend the trial of Bùi Thị Minh Hằng, NVMinh & NTQuỳnh.', 'langid': 'en'}


['hello', 'world', 'yolo']

In [48]:
print(vocab)

{'³', '흑', '╣', 'ΐ', 'ි', '…', 'ˈ', 'ж', 'ή', '命', 'á', 'ಣ', 'ď', '≠', 'ệ', 'm', 'Ι', 'ื', 'ằ', 'Х', '公', 'α', '朋', '¦', '話', '스', 'һ', 'त', '甫', '\uf8e7', '♬', 'ž', '铆', '의', '徐', '善', '\x81', '»', 'ϑ', '市', 'ِ', '隊', 'ر', 'r', 'Φ', 'ˆ', 'χ', 'К', 'ש', 'ý', 'Ê', '√', '海', '黑', 'ـ', 'ũ', 'Й', 'й', '\uf76f', 'Ό', 'ć', 'ª', '❞', '医', 'à', '止', '年', '\u202c', 'I', '±', 'ヨ', 'ˌ', '\x98', '于', '¡', '现', 'す', '―', 'ɹ', '❓', 'ņ', '→', '艾', 'ケ', 'ј', '关', '讨', 'ह', '#', 'ś', 'ă', 'Τ', 'ᾶ', 'आ', '魯', 'Χ', 'E', 'Ș', 'a', '4', '`', '选', '系', 'リ', '\uf086', 't', '6', '\uf03c', 'Ī', '島', '+', '¹', ',', '入', 'τ', 'y', 'ע', '리', 'כ', 'щ', '）', '·', '土', '鶏', 'Ú', 'l', 'Ж', 'ż', '切', '出', '"', 'к', '\uf773', '╠', 'μ', 'ّ', 'c', 's', '卡', '\uf769', '\u200f', 'ත', '샤', '业', 'ů', 'ऐ', 'ค', 'ë', '本', '片', 'ئ', '‱', 'ව', 'ш', 'ी', 'は', '芒', 'ً', '资', '中', 'с', '慢', 'H', '⇒', '的', 'Ė', 'ř', 'И', 'ễ', '™', 'Θ', 'η', '²', 'ė', 'U', 'Ł', '培', 'ア', 'Ё', '¯', 'ǒ', 'げ', '混', '给', '\x88', '(', '₹', 'ッ', 'र', '،', 

In [58]:
#Lets first get a naive bayes system based only on unigrams.
print(langind)

{'bn': 0, 'ta': 1, 'sv': 2, 'hi': 3, 'en': 4, 'fr': 5, 'mr': 6, 'ml': 7, 'it': 8, 'de': 9, 'pt': 10, 'kn': 11, 'es': 12}


In [29]:
#we get each word in the vocabulary and assign it a unique index.
wordind = {};
for i in vocab:
    wordind[i] = len(wordind)

In [46]:
wordind['$']

489

In [13]:
# print(len(wordind)) #The number of unique words in the vocabulary.

In [30]:
## Then we create a matrix of the size of the vocabulary and the number of languages.
## This denotes the frequencies of each word in the vocabulary for each of the languages.
def get_freq_matrix(data, wordind, langind):
    matrix = numpy.zeros((len(wordind), len(langind)));
    for i in range(len(data)):
        strlist = unigram(data[i]['text'])
        for j in strlist:
            matrix[wordind[j]][langind[data[i]['langid']]] += 1;
    return matrix
train_matrix = get_freq_matrix(train_data, wordind, langind)

In [15]:
def get_lang_cnts(data):
    langcnt = {};
    for i in data:
        if i['langid'] in langcnt:
            langcnt[i['langid']] += 1;
        else:
            langcnt[i['langid']] = 1;
    return langcnt
langcnt_train = get_lang_cnts(train_data)
langcnt_valid_new = get_lang_cnts(valid_new)
langcnt_valid_old = get_lang_cnts(valid_old)
def print_sorted_vals(d):
    total = 0;
    for i in d:
        total += d[i]
    print("total=", total)
    for i in sorted(d, key=d.get, reverse=True):
        print(i, "=", 100*d[i]/total, end = ",")
print("Training set language counts:")
print_sorted_vals(langcnt_train)
print("\nValidation set language counts:")
print("Old validation set:")
print_sorted_vals(langcnt_valid_old)
print("\nNew validation set:")
print_sorted_vals(langcnt_valid_new)

Training set language counts:
total= 800003
en = 39.47510196836762,fr = 10.297086385926052,de = 10.120087049673563,es = 9.813713198575506,it = 9.31634006372476,pt = 9.194090522160542,sv = 9.09934087747171,kn = 0.47349822438165856,bn = 0.4502483115688316,hi = 0.44599832750627183,ml = 0.44512333078750954,ta = 0.44462333266250254,mr = 0.424748407193473,
Validation set language counts:
Old validation set:
total= 100000
en = 39.475,fr = 10.297,de = 10.12,es = 9.814,it = 9.316,pt = 9.194,sv = 9.099,kn = 0.473,bn = 0.451,hi = 0.446,ta = 0.445,ml = 0.445,mr = 0.425,
New validation set:
total= 2446
en = 33.19705641864268,fr = 8.667211774325429,de = 8.503679476696648,es = 8.258381030253474,it = 7.849550286181521,pt = 7.726901062959935,sv = 7.686017988552739,mr = 4.170073589533933,bn = 3.0253475061324613,hi = 2.9026982829108747,ta = 2.7800490596892886,ml = 2.7800490596892886,kn = 2.4529844644317254,

In [16]:
laplace_smoothing = 0.25;

In [17]:
vec_quad = CountVectorizer(tokenizer=quadgram);
naivebayes_quad = make_pipeline(vec_quad,MultinomialNB(alpha=laplace_smoothing));
naivebayes_quad.fit(X_train, y_train)

c:\Users\tripl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\tripl\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "D:\temp\ipykernel_2568\902157978.py", line 3, in <module>
    naivebayes_quad.fit(X_train, y_train)
  File "c:\Users\tripl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py", line 1351, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\tripl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\pipeline.py", line 471, in fit
    Xt = self._fit(X, y, routed_params)
  File "c:\Users\tripl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\pipeline.py", line 408, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "c:\Users\tripl\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\memory.py", line 353, in __call__
    return self.func(*args, **kwargs)
  File "c

In [ ]:
vec_penta = CountVectorizer(tokenizer=pentagram);
naivebayes_penta = make_pipeline(vec_penta,MultinomialNB(alpha=laplace_smoothing));
naivebayes_penta.fit(X_train, y_train)

c:\Users\tripl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('countvectorizer',
                 CountVectorizer(tokenizer=<function pentagram at 0x000002179A110AF0>)),
                ('multinomialnb', MultinomialNB(alpha=0.25))])

In [ ]:
vec_word_quad = CountVectorizer(tokenizer=word_and_quadgram);
naivebayes_word_quad = make_pipeline(vec_word_quad,MultinomialNB(alpha=laplace_smoothing));
naivebayes_word_quad.fit(X_train, y_train)

c:\Users\tripl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('countvectorizer',
                 CountVectorizer(tokenizer=<function word_and_quadgram at 0x000002179AE21C60>)),
                ('multinomialnb', MultinomialNB(alpha=0.25))])

In [ ]:
vec_word_penta = CountVectorizer(tokenizer=word_and_pentagram);
naivebayes_word_penta = make_pipeline(vec_word_penta,MultinomialNB(alpha=laplace_smoothing));
naivebayes_word_penta.fit(X_train, y_train)

c:\Users\tripl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\tripl\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "D:\temp\ipykernel_3084\2669804362.py", line 3, in <module>
    naivebayes_word_penta.fit(X_train, y_train)
  File "c:\Users\tripl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py", line 1351, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\tripl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\pipeline.py", line 471, in fit
    Xt = self._fit(X, y, routed_params)
  File "c:\Users\tripl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\pipeline.py", line 408, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "c:\Users\tripl\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\memory.py", line 353, in __call__
    return self.func(*args, **kwargs)
  

In [18]:

def run_tests(clf):
    y_pred_old_valid = clf.predict(X_valid_old)
    y_pred_new_valid = clf.predict(X_valid_new)
    y_pred_train = clf.predict(X_train)
    print("accuracies: ", accuracy_score(y_valid_old, y_pred_old_valid), accuracy_score(y_valid_new, y_pred_new_valid), accuracy_score(y_train, y_pred_train))
    #print("confusion matrix: ", confusion_matrix(y_valid_old, y_pred_old_valid), confusion_matrix(y_valid_new, y_pred_new_valid), confusion_matrix(y_train, y_pred_train))
    print("micro F1  : ", compute_micro_f1_score(y_valid_old, y_pred_old_valid), compute_micro_f1_score(y_valid_new, y_pred_new_valid), compute_micro_f1_score(y_train, y_pred_train))
    print("macro F1  : ", compute_macro_f1_score(y_valid_old, y_pred_old_valid), compute_macro_f1_score(y_valid_new, y_pred_new_valid), compute_macro_f1_score(y_train, y_pred_train))
    print("-------------------------------------------------\n");
    # print("Conf matrix for valid_new");
    # conf = confusion_matrix(y_valid_new, y_pred_new_valid, labels=languages);
    # disp = ConfusionMatrixDisplay(conf, display_labels=languages);
    # disp.plot();
    # plt.show();
# run_tests(naivebayes_bi)
# run_tests(naivebayes_tri)
# run_tests(naivebayes_bi_tri)

In [51]:
def predict(clf, l):
    #l should be a numpy array
    ruled_out = set();
    #first we traverse l.
    for s in l:
        for x in s:
            if x in wordind:
                freq = train_matrix[wordind[x]];
                for i in range(len(freq)):
                    if freq[i] == 0:
                        ruled_out.add(langname[i]);
                        print("freq is 0 for ", x, ",", langname[i]);
    return ruled_out;
    

In [54]:
predict(' ', ['흑hello world'])

freq is 0 for  흑 , bn
freq is 0 for  흑 , ta
freq is 0 for  흑 , sv
freq is 0 for  흑 , hi
freq is 0 for  흑 , fr
freq is 0 for  흑 , mr
freq is 0 for  흑 , ml
freq is 0 for  흑 , it
freq is 0 for  흑 , de
freq is 0 for  흑 , pt
freq is 0 for  흑 , kn
freq is 0 for  흑 , es


{'bn', 'de', 'es', 'fr', 'hi', 'it', 'kn', 'ml', 'mr', 'pt', 'sv', 'ta'}

In [19]:
for i in range(1,4):
    vec_penta_uni = CountVectorizer(tokenizer=penta_and_unigram);
    naivebayes_penta_uni = make_pipeline(vec_penta_uni, MultinomialNB(alpha=laplace_smoothing));
    naivebayes_penta_uni.fit(X_train, y_train);
    run_tests(naivebayes_penta_uni);

accuracies:  0.98563 0.9354047424366312 0.9899350377436085
micro F1  :  0.9764834877098811 0.9070588235294118 0.9834976666782803
macro F1  :  0.9837475931142817 0.9282550051930222 0.9856807594361986
-------------------------------------------------



c:\Users\tripl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


accuracies:  0.98563 0.9354047424366312 0.9899350377436085
micro F1  :  0.9764834877098811 0.9070588235294118 0.9834976666782803
macro F1  :  0.9837475931142817 0.9282550051930222 0.9856807594361986
-------------------------------------------------



c:\Users\tripl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


accuracies:  0.98563 0.9354047424366312 0.9899350377436085
micro F1  :  0.9764834877098811 0.9070588235294118 0.9834976666782803
macro F1  :  0.9837475931142817 0.9282550051930222 0.9856807594361986
-------------------------------------------------



In [21]:
vec_uni = CountVectorizer(tokenizer=unigram);
naivebayes_uni = make_pipeline(vec_uni, MultinomialNB(alpha=laplace_smoothing));
naivebayes_uni.fit(X_train, y_train);
run_tests(naivebayes_uni);

c:\Users\tripl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


accuracies:  0.84297 0.7150449713818479 0.8419705926102777
micro F1  :  0.7511568204868154 0.5923976608187135 0.7493660998737155
macro F1  :  0.8012147270404366 0.6381383163094295 0.7999339362919269
-------------------------------------------------



In [23]:
naivebayes_uni.

{'memory': None,
 'steps': [('countvectorizer',
   CountVectorizer(tokenizer=<function unigram at 0x000001DD474431C0>)),
  ('multinomialnb', MultinomialNB(alpha=0.25))],
 'verbose': False,
 'countvectorizer': CountVectorizer(tokenizer=<function unigram at 0x000001DD474431C0>),
 'multinomialnb': MultinomialNB(alpha=0.25),
 'countvectorizer__analyzer': 'word',
 'countvectorizer__binary': False,
 'countvectorizer__decode_error': 'strict',
 'countvectorizer__dtype': numpy.int64,
 'countvectorizer__encoding': 'utf-8',
 'countvectorizer__input': 'content',
 'countvectorizer__lowercase': True,
 'countvectorizer__max_df': 1.0,
 'countvectorizer__max_features': None,
 'countvectorizer__min_df': 1,
 'countvectorizer__ngram_range': (1, 1),
 'countvectorizer__preprocessor': None,
 'countvectorizer__stop_words': None,
 'countvectorizer__strip_accents': None,
 'countvectorizer__token_pattern': '(?u)\\b\\w\\w+\\b',
 'countvectorizer__tokenizer': <function __main__.unigram(text)>,
 'countvectorizer__v

### OLD TESTS

## NEW TESTS